# ChemProp train and test model
In this script, we will train a ChemProp model and make some predictions.
We will also visualized the results

Quick startup:

https://chemprop.readthedocs.io/en/latest/quickstart.html

References:

Kevin Yang, Kyle Swanson, Wengong Jin, Connor Coley, Philipp Eiden, Hua Gao, Angel Guzman-Perez, Timothy Hopper, Brian Kelley, Miriam Mathea, Andrew Palmer, Volker Settels, Tommi Jaakkola, Klavs Jensen, and Regina Barzilay. Analyzing learned molecular representations for property prediction. Journal of Chemical Information and Modeling, 59(8):3370–3388, 2019. PMID: 31361484. URL: https://doi.org/10.1021/acs.jcim.9b00237, arXiv:https://doi.org/10.1021/acs.jcim.9b00237, doi:10.1021/acs.jcim.9b00237.

Esther Heid, Kevin P. Greenman, Yunsie Chung, Shih-Cheng Li, David E. Graff, Florence H. Vermeire, Haoyang Wu, William H. Green, and Charles J. McGill. Chemprop: a machine learning package for chemical property prediction. Journal of Chemical Information and Modeling, 64(1):9–17, 2024. PMID: 38147829. URL: https://doi.org/10.1021/acs.jcim.3c01250, arXiv:https://doi.org/10.1021/acs.jcim.3c01250, doi:10.1021/acs.jcim.3c01250.

In [1]:
# Install ChemProp - install Pytorch if you haven't already, Google Colab already has PyTorch installed.
!pip install chemprop

In [2]:
# Let's load and see our csv
import pandas as pd

input_path = '/content/drive/MyDrive/Colab_Notebooks/TRPV1-drug-discovery-research/trpv1_agonists_train_test_data.csv'
df_input = pd.read_csv(input_path)
df_input.head(5)

,canonical_smiles,pchembl_value
0,Cc1nc2cc(NC(=O)c3ccc(-c4ccc(F)cc4)nc3C)ccc2s1,7.20
1,Cc1nc2cc(NC(=O)c3ccc(-c4ccc(F)cc4)nc3C)ccc2s1,7.31
2,Cc1nc2cc(NC(=O)c3ccc(-c4ccc(F)cc4)nc3C)ccc2s1,8.05
3,Cc1nc2cc(NC(=O)c3ccc(-c4ccc(F)cc4)nc3C)ccc2s1,7.40
4,Cc1nc2cc(NC(=O)c3ccc(-c4ccc(F)cc4)nc3C)ccc2s1,7.00


### Let's train our model with ChemProp!

https://chemprop.readthedocs.io/en/latest/training.html

**Switch Google Colab environment to T4 to enable GPU computation**

In [3]:
from lightning import pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint
from chemprop import data, featurizers, models, nn

In [4]:
# Set seed for reproducibility
pl.seed_everything(42)

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


42

In [5]:
num_workers = 0  # number of workers for dataloader. 0 means using main process for data loading
smiles_column = 'canonical_smiles'
target_columns = ['pchembl_value']

In [6]:
smis = df_input.loc[:, smiles_column].values
ys = df_input.loc[:, target_columns].values

smis[:5], ys[:5]

(array(['Cc1nc2cc(NC(=O)c3ccc(-c4ccc(F)cc4)nc3C)ccc2s1',
        'Cc1nc2cc(NC(=O)c3ccc(-c4ccc(F)cc4)nc3C)ccc2s1',
        'Cc1nc2cc(NC(=O)c3ccc(-c4ccc(F)cc4)nc3C)ccc2s1',
        'Cc1nc2cc(NC(=O)c3ccc(-c4ccc(F)cc4)nc3C)ccc2s1',
        'Cc1nc2cc(NC(=O)c3ccc(-c4ccc(F)cc4)nc3C)ccc2s1'], dtype=object),
 array([[7.2 ],
        [7.31],
        [8.05],
        [7.4 ],
        [7.  ]]))

In [7]:
all_data = [data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(smis, ys)]

In [8]:
mols = [d.mol for d in all_data]  # RDkit Mol objects are use for structure based splits
train_indices, val_indices, test_indices = data.make_split_indices(mols, "random", (0.8, 0.1, 0.1))  # unpack the tuple into three separate lists
train_data, val_data, test_data = data.split_data_by_indices(
    all_data, train_indices, val_indices, test_indices
)

In [9]:
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

train_dset = data.MoleculeDataset(train_data[0], featurizer)
scaler = train_dset.normalize_targets()

val_dset = data.MoleculeDataset(val_data[0], featurizer)
val_dset.normalize_targets(scaler)

test_dset = data.MoleculeDataset(test_data[0], featurizer)

In [10]:
train_loader = data.build_dataloader(train_dset, num_workers=num_workers)
val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)
test_loader = data.build_dataloader(test_dset, num_workers=num_workers, shuffle=False)

In [11]:
mp = nn.BondMessagePassing()

In [12]:
agg = nn.MeanAggregation()

In [13]:
output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)

In [14]:
ffn = nn.RegressionFFN(output_transform=output_transform)

In [15]:
batch_norm = True

In [16]:
metric_list = [nn.metrics.RMSE(), nn.metrics.MAE()] # Only the first metric is used for training and early stopping

In [17]:
mpnn = models.MPNN(mp, agg, ffn, batch_norm, metric_list)
mpnn

MPNN(
  (message_passing): BondMessagePassing(
    (W_i): Linear(in_features=86, out_features=300, bias=False)
    (W_h): Linear(in_features=300, out_features=300, bias=False)
    (W_o): Linear(in_features=372, out_features=300, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
    (V_d_transform): Identity()
    (graph_transform): Identity()
  )
  (agg): MeanAggregation()
  (bn): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (predictor): RegressionFFN(
    (ffn): MLP(
      (0): Sequential(
        (0): Linear(in_features=300, out_features=300, bias=True)
      )
      (1): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=300, out_features=1, bias=True)
      )
    )
    (criterion): MSE(task_weights=[[1.0]])
    (output_transform): UnscaleTransform()
  )
  (X_d_transform): Identity()
  (metrics): ModuleList(
    (0): RMSE(task_weights=[[1.0]])
    (1): MAE(task_weigh

In [18]:
# Configure model checkpointing
checkpointing = ModelCheckpoint(
    "checkpoints",  # Directory where model checkpoints will be saved
    "best",  # Filename format for checkpoints, including epoch and validation loss
    "val_loss",  # Metric used to select the best checkpoint (based on validation loss)
    mode="min",  # Save the checkpoint with the lowest validation loss (minimization objective)
    save_last=True,  # Always save the most recent checkpoint, even if it's not the best
)

epochs = 60
trainer = pl.Trainer(
    logger=False,
    enable_checkpointing=True, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=epochs, # number of epochs to train for
    callbacks=[checkpointing], # Use the configured checkpoint callback
)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: 💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
INFO:lightning.pytorch.utilities.rank_zero:💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.


In [19]:
trainer.fit(mpnn, train_loader, val_loader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loading `train_dataloader` to estimate number of stepping batches.
INFO:lightning.pytorch.utilities.rank_zero:Loading `train_dataloader` to estimate number of stepping batches.


┏━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name            ┃ Type               ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ message_passing │ BondMessagePassing │  227 K │ train │     0 │
│ 1 │ agg             │ MeanAggregation    │      0 │ train │     0 │
│ 2 │ bn              │ BatchNorm1d        │    600 │ train │     0 │
│ 3 │ predictor       │ RegressionFFN      │ 90.6 K │ train │     0 │
│ 4 │ X_d_transform   │ Identity           │      0 │ train │     0 │
│ 5 │ metrics         │ ModuleList         │      0 │ train │     0 │
└───┴─────────────────┴────────────────────┴────────┴───────┴───────┘

Trainable params: 318 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 318 K                                                                                                
Total estimated model params size (MB): 1                                                                          
Modules in train mode: 25                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

INFO: `Trainer.fit` stopped: `max_epochs=60` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=60` reached.


## How good is our model?

In [20]:
# Get validating/testing MAE and RMSE
val = trainer.validate(mpnn, val_loader)
val

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val/mae          │    0.4289324879646301     │
│         val/rmse          │    0.5735898613929749     │
│         val_loss          │    0.32900533080101013    │
└───────────────────────────┴───────────────────────────┘

[{'val/rmse': 0.5735898613929749,
  'val/mae': 0.4289324879646301,
  'val_loss': 0.32900533080101013}]

In [21]:
# Get testing MAE and RMSE
trainer.test(dataloaders=test_loader, weights_only=False)

/usr/local/lib/python3.12/dist-packages/lightning/pytorch/trainer/connectors/checkpoint_connector.py:149: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
INFO: Restoring states from the checkpoint path at /content/checkpoints/best.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at /content/checkpoints/best.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at /content/checkpoints/best.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at /content/checkpoints/best.ckpt


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/mae          │    0.45988184213638306    │
│         test/rmse         │    0.6068782806396484     │
└───────────────────────────┴───────────────────────────┘

[{'test/rmse': 0.6068782806396484, 'test/mae': 0.45988184213638306}]

### My interpretation:

Our model's Mean Absolute Error (MAE) of 0.42 (training). This means our average prediction will be off by less than 1 unit. Another way of saying this, is that our prediction will be off by about 0.42.

For example, a prediction of 9 could be more or less than 0.42.

Of course, there is still room for optimization such as increasing the number of epochs in training or providing a larger model dataset.

For now, I think this is a good start in finding potential TRPV1 agonists.

**Let's now save our model to use for future predictions**

In [22]:
# End script here
assert False

AssertionError: 

In [23]:
# Run this cell to downlaod the best model file
from google.colab import files

"Download the .ckpt model file"
files.download(f'/content/checkpoints/best.ckpt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>